In [28]:
import pandas as pd 
import json
import os
import shutil
import dask.dataframe as dd
from dask.distributed import Client

In [29]:
client = Client(memory_limit='2GB', memory_target_fraction=0.6, local_directory='/path/to/directory')

c:\Users\eloua\AppData\Local\Programs\Python\Python313\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49866 instead
  warnings.warn(


# Chargement des data sets

In [ ]:
parquet_file = "../data/final_output.parquet"
data = dd.read_parquet(parquet_file, blocksize='25MB')
# data = data.head(2000, compute=False) # pour tester

# Traitement des données

In [31]:
def add_action_reverse(df):
    df["action_reverse"] = df.groupby("session").cumcount()
    df["action_reverse"] = df.groupby("session")["action_reverse"].apply(lambda x: x.max() - x ).reset_index(drop=True) #, meta=('action_reverse', 'int32'))
    return df

def add_session_length(df):
    df['session_length'] = df.groupby('session')['session'].transform('count') #, meta=('session_length', 'int32'))
    return df

def add_log_recency_score(df):
    linear_interpolation = 0.1 + ((1 - 0.1) / (df['session_length'] - 1)) * (df['session_length'] - df['action_reverse'] - 1)
    df['log_recency_score'] = (2 ** linear_interpolation) - 1
    df['log_recency_score'] = df['log_recency_score'].fillna(1)
    return df

def add_type_weighted_log_recency_score(df):
    type_weights = {0: 1, 1: 6, 2: 3}
    df['type_weighted_log_recency_score'] = df['log_recency_score'] / df['type'].map(type_weights)
    return df

def apply_pipeline(df, pipeline):
    if isinstance(df, (dd.DataFrame, pd.DataFrame)):
        for f in pipeline:
            df = f(df)
        return df
    else:
        raise TypeError("Input doit être un DataFrame Pandas ou Dask DataFrame")
    
def process_partition(partition):
    type_mapping = {
        'clicks': 0,
        'carts': 1,
        'orders': 2
    }
    partition['type'] = partition['type'].map(type_mapping)
    partition['type'] = partition['type'].astype('int8')
    partition = apply_pipeline(partition, pipeline)
    expected_columns = ['session', 'action_reverse', 'session_length', 'log_recency_score', 
                        'type_weighted_log_recency_score', 'aid', 'ts', 'type']
    missing_columns = [col for col in expected_columns if col not in partition.columns]
    if missing_columns:
        raise ValueError(f"Colonnes manquantes après transformation: {missing_columns}")
    return partition[expected_columns] 

In [32]:
pipeline = [add_action_reverse, add_session_length, add_log_recency_score, add_type_weighted_log_recency_score]

In [33]:
meta = {
    'session': 'int32',
    'action_reverse': 'int32',
    'session_length': 'int32',
    'log_recency_score': 'float32',
    'type_weighted_log_recency_score': 'float32',
    'aid': 'int32',
    'ts': 'int32',
    'type': 'int8'
}

In [34]:
df_processed = data.map_partitions(process_partition, meta=meta)

In [35]:
output = df_processed.compute()

In [36]:
output.to_parquet('dataframe.parquet', engine='pyarrow')

In [ ]:
data_2 = dd.read_parquet('dataframe.parquet')

2000